In [ ]:
SERVER_PORT=49052
SERVER_HOST='0.0.0.0'
DATABASE_PORT=27017

import datetime
from bson.objectid import ObjectId
import random
import string
import hmac
import hashlib
from hashlib import sha256
from flask import Flask
from flask import request,redirect
from flask import Response
from flask import render_template
from os import listdir
from os.path import isfile, join
import os
from pymongo import MongoClient
import pymongo
import os.path
from base64 import b64encode
from collections import OrderedDict
from hashlib import sha256
from hmac import HMAC
from urllib.parse import urlparse, parse_qsl, urlencode
import json

app = Flask(__name__)
app.config['TEMPLATES_AUTO_RELOAD'] = True

@app.after_request
def add_header(r):
    r.headers["Cache-Control"] = "no-cache, no-store, must-revalidate"
    r.headers["Pragma"] = "no-cache"
    r.headers["Expires"] = "0"
    r.headers['Cache-Control'] = 'public, max-age=0'
    return r

@app.route('/',methods=["POST"])
def command():
    t=request.json
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    appID=t["appID"]
    appObject=db.apps.find_one({"_id":ObjectId(appID)})
    print(t)
    tSign=b64encode(HMAC(appObject["secretKey"].encode(), t["commandHash"].encode(), sha256).digest())
    print(tSign)
    if t["appURL"]!=appObject["appSite"]:
        return "False"
    if tSign.decode("utf-8")==t["serverSign"]:
        allowedCommands=list()
        for i in ["spisok","read","write","delete"]:
            if appObject[i]=="on":
                allowedCommands.append(i)
        print(allowedCommands)
        return str(t["type"] in allowedCommands)
    else:
        return "False"

app.run(host=SERVER_HOST, port=SERVER_PORT, threaded=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:49052/ (Press CTRL+C to quit)
127.0.0.1 - - [03/May/2021 14:11:01] "POST / HTTP/1.1" 200 -


{'appID': '608f1351b59c6f34109189d9', 'type': 'read', 'commandHash': '2cdaeb6d48b5ccd8d6666cf725fa86eaa4cf7aacb172dcdb9082e466af42d94a', 'serverSign': 'NVuGAH4oBH5OBXSIR5kf+W8/xez2WpyuRID2rsfxPLs=', 'appURL': 'http://localhost:49054/'}
b'NVuGAH4oBH5OBXSIR5kf+W8/xez2WpyuRID2rsfxPLs='
['read', 'write']


127.0.0.1 - - [03/May/2021 14:19:49] "POST / HTTP/1.1" 200 -


{'appID': '608f1351b59c6f34109189d9', 'type': 'write', 'commandHash': '1aec2e1ea3a8cc4d34b1c88c12921c75b6de4be89bffe836de084da940e908ba', 'serverSign': 'WJ668iphJ2IepmDk95SF5fvHEUiMfsYjbWOVZ9kQfc8=', 'appURL': 'http://localhost:49054/'}
b'WJ668iphJ2IepmDk95SF5fvHEUiMfsYjbWOVZ9kQfc8='
['read', 'write']


...

{'appID': '608f1351b59c6f34109189d9', 'type': 'read', 'commandHash': '2cdaeb6d48b5ccd8d6666cf725fa86eaa4cf7aacb172dcdb9082e466af42d94a', 'serverSign': 'NVuGAH4oBH5OBXSIR5kf+W8/xez2WpyuRID2rsfxPLs=', 'appURL': 'http://localhost:49054/'}
b'NVuGAH4oBH5OBXSIR5kf+W8/xez2WpyuRID2rsfxPLs='
['read', 'write']


127.0.0.1 - - [03/May/2021 15:18:28] "POST / HTTP/1.1" 200 -


{'appID': '608f1351b59c6f34109189d9', 'type': 'write', 'commandHash': '9cadb22661ac5567c4a45bed000b2c3cf2f106f031cc31f1d778cc0f776e28fd', 'serverSign': 'j/iunq4gW4ifcyCBVVOOYb+dZMKHvZ4NK40dwZhtsGk=', 'appURL': 'http://localhost:49054/'}
b'j/iunq4gW4ifcyCBVVOOYb+dZMKHvZ4NK40dwZhtsGk='
['read', 'write']


127.0.0.1 - - [03/May/2021 15:18:35] "POST / HTTP/1.1" 200 -


{'appID': '608f1351b59c6f34109189d9', 'type': 'read', 'commandHash': '2cdaeb6d48b5ccd8d6666cf725fa86eaa4cf7aacb172dcdb9082e466af42d94a', 'serverSign': 'NVuGAH4oBH5OBXSIR5kf+W8/xez2WpyuRID2rsfxPLs=', 'appURL': 'http://localhost:49054/'}
b'NVuGAH4oBH5OBXSIR5kf+W8/xez2WpyuRID2rsfxPLs='
['read', 'write']


In [2]:
from urllib.parse import urlparse
urlparse("http://localhost:49054/")

ParseResult(scheme='http', netloc='localhost:49054', path='/', params='', query='', fragment='')

In [16]:
import json
heh=json.loads('{"command":{"type": "spisok"},"sign":"RFAhWxAZ2Z4dodrihFiY29IkGa5+nkMMtZPvC1ZQRr8="}')

In [17]:
heh

{'command': {'type': 'spisok'},
 'sign': 'RFAhWxAZ2Z4dodrihFiY29IkGa5+nkMMtZPvC1ZQRr8='}

In [18]:
heh.items()

dict_items([('command', {'type': 'spisok'}), ('sign', 'RFAhWxAZ2Z4dodrihFiY29IkGa5+nkMMtZPvC1ZQRr8=')])

In [11]:
type(heh)

dict

In [14]:
heh=json.dumps(heh)
heh

'{"command": {"type": "spisok"}, "sign": "RFAhWxAZ2Z4dodrihFiY29IkGa5+nkMMtZPvC1ZQRr8="}'

In [15]:
from base64 import b64encode
from collections import OrderedDict
from hashlib import sha256
from hmac import HMAC
from urllib.parse import urlparse, parse_qsl, urlencode
od=OrderedDict(sorted(heh.items(), key=lambda x: x[0]))
hash_code =b64encode(HMAC("0c863557e6e6cfe5b859d19f83b6add9c42bfe9fc1d696f36dd00da7bb523a59".encode(), urlencode(od, doseq=True).encode(), sha256).digest())
hash_code.decode('utf-8')

AttributeError: 'str' object has no attribute 'items'

In [3]:
type("lol")

str

In [7]:
b'Ad/tTn0cLI/rGz+z6p0CUyG/wUcTr/m18fV6C0nYP74='.decode("utf-8")

'Ad/tTn0cLI/rGz+z6p0CUyG/wUcTr/m18fV6C0nYP74='